<a href="https://colab.research.google.com/github/jostz0/Sentimentanalyse-Fiskalpolitischer-Wirtschaftsartikel/blob/main/scoring_modell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m spacy download de_core_news_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 MB 17.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import pandas as pd
import spacy

# Laden des vortrainierten spaCy-Modells (für Deutsch)
nlp = spacy.load('de_core_news_md')

# Einlesen der expansiven und kontraktiven Begriffe
expansionary_terms = pd.read_csv('expansionary_terms_preprocessed.csv', header=None)[0].tolist()
contractionary_terms = pd.read_csv('contractionary_terms_preprocessed.csv', header=None)[0].tolist()

print("Anzahl der expansiven Begriffe:", len(expansionary_terms))
print("Anzahl der kontraktiven Begriffe:", len(contractionary_terms))

Anzahl der expansiven Begriffe: 123
Anzahl der kontraktiven Begriffe: 97


**Alternative: Diesen Code nur ausführen, wenn die bestehenden Listen um Synonyme erweitert werden sollen!**

In [ ]:
# Funktion zur Ermittlung ähnlicher Begriffe und Erstellung einer Zuordnungstabelle
def find_similar_terms_mapping(term_list, n=5):
    term_mapping = {}
    for term in term_list:
        token = nlp(term)
        if token.has_vector and token.vector_norm > 0:  # Überprüfen, ob der Originalbegriff einen Vektor hat
            similar_terms = set()
            for word in nlp.vocab:
                # Überprüfen, ob das Wort einen gültigen Vektor hat und nicht denselben Begriff repräsentiert
                if word.is_alpha and word.has_vector and word.vector_norm > 0 and word.text != term:
                    similarity = token.similarity(word)
                    if similarity > 0.7:  # Schwellenwert für Ähnlichkeit
                        similar_terms.add(word.text.lower())
            term_mapping[term] = list(similar_terms)[:n]  # Maximal n Synonyme aufnehmen
    return term_mapping

# Funktion zum Entfernen von Synonymen, die Wörter aus den Originalbegriffen enthalten
def remove_duplicate_synonyms(df):
    for i, row in df.iterrows():
        original_terms = set(row['Original Term'].split())
        for col in df.columns[1:]:
            if pd.notna(row[col]):  # Prüfen, ob das Synonym vorhanden ist
                synonym_words = set(row[col].split())
                # Entfernen, wenn das Synonym ein Wort enthält, das im Originalterm vorkommt
                if original_terms & synonym_words:
                    df.at[i, col] = None
    return df

# Erstellen der Zuordnungstabellen für die expansiven und kontraktiven Begriffe
expanded_expansionary_mapping = find_similar_terms_mapping(expansionary_terms)
expanded_contractionary_mapping = find_similar_terms_mapping(contractionary_terms)

# Konvertieren der Zuordnungen in DataFrames
expansionary_df = pd.DataFrame.from_dict(expanded_expansionary_mapping, orient='index').reset_index()
expansionary_df.columns = ['Original Term'] + [f'Synonym {i+1}' for i in range(expansionary_df.shape[1] - 1)]

contractionary_df = pd.DataFrame.from_dict(expanded_contractionary_mapping, orient='index').reset_index()
contractionary_df.columns = ['Original Term'] + [f'Synonym {i+1}' for i in range(contractionary_df.shape[1] - 1)]

# Anwenden der Funktion zur Bereinigung von doppelten Synonymen
expansionary_df = remove_duplicate_synonyms(expansionary_df)
contractionary_df = remove_duplicate_synonyms(contractionary_df)

# Extrahieren der erweiterten Listen für das Scoring
expanded_expansionary_terms = set(expansionary_terms)
for synonyms in expansionary_df.iloc[:, 1:].values.flatten():
    if pd.notna(synonyms):
        expanded_expansionary_terms.add(synonyms)

expanded_contractionary_terms = set(contractionary_terms)
for synonyms in contractionary_df.iloc[:, 1:].values.flatten():
    if pd.notna(synonyms):
        expanded_contractionary_terms.add(synonyms)

# Speichern der DataFrames in CSV-Dateien
expansionary_df.to_csv('expanded_expansionary_terms.csv', index=False, encoding='utf-8')
contractionary_df.to_csv('expanded_contractionary_terms.csv', index=False, encoding='utf-8')

print(f"Anzahl der erweiterten expansiven Begriffe: {len(expanded_expansionary_terms)}")
print(f"Anzahl der erweiterten kontraktiven Begriffe: {len(expanded_contractionary_terms)}")

Anzahl der erweiterten expansiven Begriffe: 133
Anzahl der erweiterten kontraktiven Begriffe: 105


Dieser Code liest die vorher bereinigten Artikel ein und teilt den Text in die einzelnen Artikel auf. Außerdem extrahiert er das Erscheinungsdatum des Artikels.

In [ ]:
# Lesen und Segmentieren der Artikel
with open('bereinigt_spezifisch.txt', 'r', encoding='utf-8') as file:
    text = file.read()

articles = text.split('handelsblatt prinen')[1:]  # Segmentierung ab dem ersten Kennzeichen
articles = ['handelsblatt prinen' + article for article in articles]

print("Anzahl der Artikel:", len(articles))

# Extrahieren des Datums aus den Artikeln
def extract_date(article):
    # Angenommen, das Datum steht direkt nach "handelsblatt prinen "
    date_marker = article.split()[2]
    # Formatierung in DD.MM.YYYY
    if len(date_marker) == 8:  # Überprüfung, ob das Datum in der erwarteten Form ist
        formatted_date = f"{date_marker[:2]}.{date_marker[2:4]}.{date_marker[4:]}"
        return formatted_date

Anzahl der Artikel: 137


Hier bestimmen wir den Score für jeden Artikel und erstellen eine Tabelle, in der wir die Scores und zugehörigen Artikel auslesen können.

**Ohne Verwendung von Synonymen!**

In [ ]:
# Funktion zum Zählen der Begriffe in einem Text
def count_terms(text, term_list):
    words = text.split()
    return sum(1 for word in words if word in term_list)

article_scores = []
for article in articles:
    date = extract_date(article)
    expansive_count = count_terms(article, expansionary_terms)
    contractionary_count = count_terms(article, contractionary_terms)
    score = expansive_count + contractionary_count
    normalized_score = score / len(article.split()) if len(article.split()) > 0 else 0  # Normalisierung des Scores
    article_scores.append({
        'Datum': date,
        'Vorschau': article[:100],
        'Expansive Begriffe': expansive_count,
        'Kontraktive Begriffe': contractionary_count,
        'Score': normalized_score
    })

# Beispielhafte Ausgabe der Scores
for i, article_data in enumerate(article_scores[:5]):  # Zeige die ersten 5 Artikel
    print(f"Artikel {i+1}:")
    print(f"Vorschau: {article_data['Vorschau']}...")
    print(f"Expansive Begriffe: {article_data['Expansive Begriffe']}, Kontraktive Begriffe: {article_data['Kontraktive Begriffe']}, Score: {article_data['Score']}\n")

# Erstellen eines DataFrames für die Ausgabe
scores_df = pd.DataFrame(article_scores)

# Speichern der Ergebnisse in einer CSV-Datei
scores_df.to_csv('artikel_scores.csv', index=False, encoding='utf-8')

Artikel 1:
Vorschau: handelsblatt prinen 28102022 s 10 steuerschätzung rekordeinnahme krise endgueltig bestaetigen handel...
Expansive Begriffe: 5, Kontraktive Begriffe: 6, Score: 0.03333333333333333

Artikel 2:
Vorschau: handelsblatt prinen 17042023 s 4 special gita gopinath groebsten usspitzenoekonomin finanzsystem rue...
Expansive Begriffe: 9, Kontraktive Begriffe: 2, Score: 0.022821576763485476

Artikel 3:
Vorschau: handelsblatt prinen 16022023 s 7 des stabilitätspakts lindner europaeisch schuldenregel retten suede...
Expansive Begriffe: 0, Kontraktive Begriffe: 13, Score: 0.04609929078014184

Artikel 4:
Vorschau: handelsblatt prinen 24062022 s 58 wochenende valdis dombrovskis periode billig geld entgegen taumeln...
Expansive Begriffe: 3, Kontraktive Begriffe: 5, Score: 0.02056555269922879

Artikel 5:
Vorschau: handelsblatt prinen 17062022 s 3 special eurokrise ezbkrisensitzung erster europaeisch finanzministe...
Expansive Begriffe: 0, Kontraktive Begriffe: 3, Score: 0.005309734513

**Alternative: Diesen Code nur ausführen, wenn die expansiven und kontraktiven Begriffe um Synonyme erweitert wurden!**

In [ ]:
# Funktion zum Zählen der Begriffe in einem Text
def count_terms(text, term_list):
    words = text.split()
    return sum(1 for word in words if word in term_list), len(words)

# Berechnung der Scores für die Artikel
article_scores_sy = []
for article in articles:
    date = extract_date(article)
    expansive_count_sy, total_words = count_terms(article, expanded_expansionary_terms)
    contractionary_count_sy, _ = count_terms(article, expanded_contractionary_terms)
    score_sy = expansive_count_sy + contractionary_count_sy
    normalized_score_sy = score_sy / total_words if total_words > 0 else 0  # Vermeidung von Division durch Null
    article_scores_sy.append({
        'Datum': date,
        'Vorschau': article[:100],
        'Expansive Begriffe': expansive_count_sy,
        'Kontraktive Begriffe': contractionary_count_sy,
        'Score': normalized_score_sy
    })

# Beispielhafte Ausgabe der Scores
for i, article_data in enumerate(article_scores_sy[:5]):  # Zeige die ersten 5 Artikel
    print(f"Artikel {i+1}:")
    print(f"Vorschau: {article_data['Vorschau']}...")
    print(f"Expansive Begriffe: {article_data['Expansive Begriffe']}, Kontraktive Begriffe: {article_data['Kontraktive Begriffe']}, Score: {article_data['Score']}\n")

# Erstellen eines DataFrames für die Ausgabe
scores_sy_df = pd.DataFrame(article_scores_sy)

# Speichern der Ergebnisse in einer CSV-Datei
scores_sy_df.to_csv('artikel_scores_sy.csv', index=False, encoding='utf-8')

Artikel 1:
Vorschau: handelsblatt prinen 28102022 s 10 steuerschätzung rekordeinnahme krise endgueltig bestaetigen handel...
Expansive Begriffe: 8, Kontraktive Begriffe: 9, Score: 0.051515151515151514

Artikel 2:
Vorschau: handelsblatt prinen 17042023 s 4 special gita gopinath groebsten usspitzenoekonomin finanzsystem rue...
Expansive Begriffe: 15, Kontraktive Begriffe: 8, Score: 0.04771784232365145

Artikel 3:
Vorschau: handelsblatt prinen 16022023 s 7 des stabilitätspakts lindner europaeisch schuldenregel retten suede...
Expansive Begriffe: 4, Kontraktive Begriffe: 13, Score: 0.06028368794326241

Artikel 4:
Vorschau: handelsblatt prinen 24062022 s 58 wochenende valdis dombrovskis periode billig geld entgegen taumeln...
Expansive Begriffe: 7, Kontraktive Begriffe: 7, Score: 0.03598971722365039

Artikel 5:
Vorschau: handelsblatt prinen 17062022 s 3 special eurokrise ezbkrisensitzung erster europaeisch finanzministe...
Expansive Begriffe: 3, Kontraktive Begriffe: 4, Score: 0.01238938053